# Regression of Vote Characteristics on CAR

In [89]:
import stata_setup

stata_setup.config('/Applications/StataNow', 'mp')

In [90]:
%%stata

clear all
set more off
version 18

* Paths
global PROCESSED_DATA "processed_data"
global TABLES "tables"
global EVENT_WINDOW 5


. 
. clear all

. set more off

. version 18

. 
. * Paths
. global PROCESSED_DATA "processed_data"

. global TABLES "tables"

. global EVENT_WINDOW 5

. 


In [91]:
%%stata
foreach stage in created end {
    import delimited using "$PROCESSED_DATA/event_study_panel_`stage'.csv", clear

    * Parse date
    gen day = date(date, "YMD")
    format day %td
    gen year = year(day)

    * Keep CAR at the end of event window
    keep if index==$EVENT_WINDOW

    * Encode categorical variables
    encode gecko_id, gen(token)
    encode space, gen(dao)

    * Encode vote characteristics
    local vars half_vp_ratio vn_hhi vs_hhi cci

    eststo clear
    foreach v of local vars {
        reghdfe car `v', absorb(dao year) vce(cluster dao)
        eststo `v'
    }

    * optional table
    esttab half_vp_ratio vn_hhi vs_hhi cci, se star(* 0.10 ** 0.05 *** 0.01)

    * save results
    esttab using "$TABLES/reg_car_votes_`stage'.tex", replace se star(* 0.10 ** 0.05 *** 0.01) ///
        label mtitles("CAR[-5,5]" "CAR[-5,5]" "CAR[-5,5]" "CAR[-5,5]")
}


. foreach stage in created end {
  2.     import delimited using "$PROCESSED_DATA/event_study_panel_`stage'.csv"
> , clear
  3. 
.     * Parse date
.     gen day = date(date, "YMD")
  4.     format day %td
  5.     gen year = year(day)
  6. 
.     * Keep CAR at the end of event window
.     keep if index==$EVENT_WINDOW
  7. 
.     * Encode categorical variables
.     encode gecko_id, gen(token)
  8.     encode space, gen(dao)
  9. 
.     * Encode vote characteristics
.     local vars half_vp_ratio vn_hhi vs_hhi cci
 10. 
.     eststo clear
 11.     foreach v of local vars {
 12.         reghdfe car `v', absorb(dao year) vce(cluster dao)
 13.         eststo `v'
 14.     }
 15. 
.     * optional table
.     esttab half_vp_ratio vn_hhi vs_hhi cci, se star(* 0.10 ** 0.05 *** 0.01)
 16. 
.     * save results
.     esttab using "$TABLES/reg_car_votes_`stage'.tex", replace se star(* 0.10 
> ** 0.05 *** 0.01) ///
>         label mtitles("CAR[-5,5]" "CAR[-5,5]" "CAR[-5,5]" "CAR[-5,5]")
 17. }
